# 10.3 多 Agent 研究系统

本笔记本构建一个完整的 Researcher + Writer + Reviewer 系统。

**预估成本**: ~$0.20

In [ ]:
!pip install openai requests -q

In [ ]:
import os
from openai import OpenAI

client = OpenAI()
print('✓ Client initialized')

## Step 1: Define Shared Memory
<!-- Step 1: 定义 Shared Memory -->

In [ ]:
class SharedMemory:
    def __init__(self):
        self.data = {
            'topic': None,
            'research_findings': None,
            'draft_version': 0,
            'current_draft': None,
            'review_feedback': [],
        }
    
    def set(self, key, value):
        self.data[key] = value
        print(f'📝 Memory updated: {key}')
    
    def get(self, key):
        return self.data.get(key)
    
    def increment_version(self):
        self.data['draft_version'] += 1
        return self.data['draft_version']

shared_memory = SharedMemory()
print('✓ Shared memory initialized')

## Step 2: 创建 Researcher Agent

In [ ]:
researcher = client.beta.assistants.create(
    name='Research Specialist',
    instructions='''Research the topic thoroughly.
Save findings and hand off to writer.''',
    model='gpt-4o-mini',
    tools=[],
    handoffs=[{'type': 'handoff', 'target': 'writer'}],
)

print(f'Researcher: {researcher.id}')

## Step 3: 创建 Writer Agent

In [ ]:
writer = client.beta.assistants.create(
    name='Content Writer',
    instructions='''Write a well-structured report.
Save draft and hand off to reviewer.''',
    model='gpt-4o-mini',
    tools=[],
    handoffs=[{'type': 'handoff', 'target': 'reviewer'}],
)

print(f'Writer: {writer.id}')

## Step 4: 创建 Reviewer Agent

In [ ]:
reviewer = client.beta.assistants.create(
    name='Quality Reviewer',
    instructions='''Review the draft and score it.
If score >= 8: hand off to coordinator.
If score < 8: hand off back to writer for revision.''',
    model='gpt-4o-mini',
    tools=[],
    handoffs=[
        {'type': 'handoff', 'target': 'writer'},
        {'type': 'handoff', 'target': 'coordinator'},
    ],
)

print(f'Reviewer: {reviewer.id}')

## Step 5: 模拟系统Run
<!-- Step 5: 模拟系统运行 -->

In [ ]:
def simulate_research_system(topic):
    print('=== 研究系统运行 ===')
    print(f'Topic: {topic}\n')
    
    print('[Step 1] Coordinator → Researcher')
    shared_memory.set('topic', topic)
    
    print('[Step 2] Researcher 搜索资料')
    research = 'Research findings about ' + topic
    shared_memory.set('research_findings', research)
    print('  Handoff to writer\n')
    
    print('[Step 3] Writer 撰写初稿')
    shared_memory.increment_version()
    draft_v1 = 'Draft v1 based on research'
    shared_memory.set('current_draft', draft_v1)
    print('  Handoff to reviewer\n')
    
    print('[Step 4] Reviewer 评分: 7/10')
    feedback = 'Needs more data and better structure'
    shared_memory.data['review_feedback'].append({'score': 7, 'feedback': feedback})
    print(f'  Feedback: {feedback}')
    print('  Handoff back to writer\n')
    
    print('[Step 5] Writer 修改')
    shared_memory.increment_version()
    draft_v2 = 'Draft v2 - improved'
    shared_memory.set('current_draft', draft_v2)
    print('  Handoff to reviewer\n')
    
    print('[Step 6] Reviewer 评分: 9/10')
    shared_memory.data['review_feedback'].append({'score': 9, 'feedback': 'Approved'})
    print('  Approved! Handoff to coordinator\n')
    
    print('[Step 7] Coordinator 返回最终报告')
    
    return {
        'status': 'success',
        'draft_version': shared_memory.get('draft_version'),
        'review_rounds': len(shared_memory.get('review_feedback')),
        'final_draft': shared_memory.get('current_draft'),
    }

result = simulate_research_system('AI Agent 市场趋势 2026')
print('\n=== 结果 ===')
for k, v in result.items():
    print(f'{k}: {v}')

## Step 6: 可视化协作流程

In [ ]:
def visualize_workflow():
    print('=== 工作流程可视化 ===')
    print()
    print('User')
    print('  ↓')
    print('Coordinator')
    print('  ↓')
    print('Researcher (搜索资料)')
    print('  ↓')
    print('Writer (撰写 v1)')
    print('  ↓')
    print('Reviewer (评分: 7/10)')
    print('  ↓ (需要修改)')
    print('Writer (撰写 v2)')
    print('  ↓')
    print('Reviewer (评分: 9/10)')
    print('  ↓ (通过)')
    print('Coordinator')
    print('  ↓')
    print('User (最终报告)')

visualize_workflow()

## 总结

你已经掌握了多 Agent 系统的完整开发！

### 关键组件
- **Shared Memory**: 所有 Agent 共享状态
- **Handoff**: 任务在 Agent 间流转
- **质量门禁**: Reviewer 控制迭代
- **防止无限循环**: 最大迭代次数限制

### 设计经验
- ✓ 单一职责：每个 Agent 专注一件事
- ✓ 明确的 Handoff 条件
- ✓ 共享内存避免重复传递
- ✓ 设置上限防止成本失控
- ✓ 详细日志便于调试

第 10 章完成！下一章学习协议篇。